# Assignment 2: Simple Neural Network - gradient 적용 

1. 이번에 사용할 자료는 과제 2에서 좀 더 발전 시킨 SimpleNetData2.csv 로 10000*201 배열로 되어 있다. 이 행렬의    마지막 열은 이전과 마찬가지로 0-9 사이의 정답 숫자로 되어 있다. .
2. 과제2를 발전시켜 교재 137페이지에 있는 class TwoLayerNet을 그대로 사용하라. 이 class를 위한 관련 모듈과      함수는 import 하지 말고 한 프로그램에 같이 명시하라 (softmax, gradient 등)  
3. load_csv 모듈은 이제 SimpleNetData2.csv 자료를 80%는 training으로 20%는 test자료로 랜덤하게 분할하도록 한다

In [2]:
import sys, os
sys.path.append(os.pardir)
from csv import reader
import numpy as np 

In [3]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [4]:
def cross_entropy_error(y, t):

    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)        

    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)   
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [5]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    #def gradient(self, x, t):
         #교재의 numerical_gradient는 사용하지 말 것.
            
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
 
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads
 

In [6]:
# Load a CSV file
def load_csv(filename):
    from random import randrange
    x_train = list()
    t_train = list() 
    x_test = list()
    t_test = list() 
    
    '''(여기에 코드 작성)
    읽어 온 자료를 numpy 배열로 바꾸고 이 때 x_train과 y_train은 각각 float형과 int형으로 한다. 80%를 훈련자료로, 20%를 시험자료로 나누고  훈련자료와 정답을 분리
    참고: 강의 자료 중 
    How to Implement Simple Linear Regression From Scratch with Python 참조'''
      
    # 데이터 list 선언
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        # row별로 csv파일을 읽어서 np.array변환후 dataset에 넣는다
        for row in csv_reader:
            dataset.append(np.array(row))
    
    # train_size를 전체 dataset의 80%로 지정
    train_size = len(dataset)*0.8
    
    # 80%의 데이터가 뽑힐때까지 while문을 돌면서 random하게 row를 뽑아 train데이터로 할당
    while len(x_train) < train_size:
        index = randrange(len(dataset))
        data = dataset.pop(index)
        # 뽑힌 row데이터를 x와 t로 나누어서 list에 넣는다
        # x데이터는 float으로, t데이터는 int로 변환해서 넣는다
        x_train.append(data[:-1].astype(np.float))
        t_train.append(data[-1].astype(np.int))
    
    # 남은 dataset에서 다시 random하게 row를 뽑아서 test데이터로 할당
    while len(dataset) > 0:
        index = randrange(len(dataset))
        data = dataset.pop(index)
         # 뽑힌 row데이터를 x와 t로 나누어서 list에 넣는다
        # x데이터는 float으로, t데이터는 int로 변환해서 넣는다
        x_test.append(data[:-1].astype(np.float))
        t_test.append(data[-1].astype(np.int))

    # 80% / 20% 로나뉜 np.array list를 return 한다.
    return np.array(x_train), np.array(t_train), np.array(x_test), np.array(t_test)

In [7]:
def one_hot_encoding(x):
    # t_train, t_test 정답 값을 one-hot encoding으로 변형. sklearn, pandas, keras 등에서 제공하는 모듈을 쓰지말고 numpy로 구현해 볼 것  
    
    # 전체 값들이 0으로 채워진 np.array를 선언한다 
    result = np.zeros((len(x),10))
    # 각각의 데이터에 해당하는 index의 값을 1로 변환
    for i in range(len(x)):
        result[i][x[i]] = 1
    return result     

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [12]:
filename = 'SimpleNetData2.csv'
x_train, t_train, x_test, t_test = load_csv(filename)
#print(t_test)
print(x_train)
print(len(x_train))
print(len(t_train))
print(len(x_test))
print(len(t_test))

[[0.       0.       0.       ... 0.827451 0.788235 0.34902 ]
 [0.607843 0.996078 0.145098 ... 0.       0.       0.      ]
 [0.       0.       0.176471 ... 0.952941 0.537255 0.415686]
 ...
 [0.       0.       0.       ... 0.       0.352941 0.94902 ]
 [0.509804 0.       0.       ... 0.       0.       0.      ]
 [0.       0.       0.262745 ... 0.992157 0.992157 0.992157]]
8000
8000
2000
2000


In [181]:
t_train = one_hot_encoding(t_train)
t_test = one_hot_encoding(t_test)
#network = TwoLayerNet(input_size=200, hidden_size=50, output_size=10)

In [183]:
np.shape(x_train)
np.shape(t_train)

(8000, 10)

In [185]:
network = TwoLayerNet(input_size=200, hidden_size=50, output_size=10)
# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1
 
train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)
 
for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
 
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
 
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
 
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
 
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.09875, 0.0965
train acc, test acc | 0.111375, 0.1245
train acc, test acc | 0.111375, 0.1245
train acc, test acc | 0.2665, 0.258
train acc, test acc | 0.295, 0.318
train acc, test acc | 0.395375, 0.4105
train acc, test acc | 0.45775, 0.458
train acc, test acc | 0.4975, 0.507
train acc, test acc | 0.542875, 0.5545
train acc, test acc | 0.57325, 0.5785
train acc, test acc | 0.594375, 0.608
train acc, test acc | 0.6135, 0.622
train acc, test acc | 0.629, 0.6365
train acc, test acc | 0.645, 0.655
train acc, test acc | 0.653375, 0.6545
train acc, test acc | 0.663125, 0.665
train acc, test acc | 0.672, 0.675
train acc, test acc | 0.6765, 0.679
train acc, test acc | 0.685875, 0.691
train acc, test acc | 0.68775, 0.687
train acc, test acc | 0.689, 0.692
train acc, test acc | 0.700375, 0.699
train acc, test acc | 0.699375, 0.699
train acc, test acc | 0.708625, 0.707
train acc, test acc | 0.7085, 0.7
train acc, test acc | 0.71525, 0.7155
train acc, test acc | 0.717125, 0.7